In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from time import sleep

driver = webdriver.Chrome()

driver.get("https://blackrhino.club/#/login")

mobile_input = driver.find_element(By.XPATH, "//input[@placeholder='Mobile Number']")
password_input = driver.find_element(By.XPATH, "//input[@placeholder='Password']")
login_button = driver.find_element(By.XPATH, "//button[contains(text(),'Login')]")

MOBILE_NUMBER = "user_name"
PASSWORD = "password"

mobile_input.send_keys(MOBILE_NUMBER)
password_input.send_keys(PASSWORD)
login_button.click()

sleep(5)

driver.get("https://blackrhino.club/#/win")
sleep(5) 

data = [] 
try:
    while True: 
        print("Scraping current page...")
        
        try:
            table = driver.find_element(By.XPATH, "(//table)[1]")  
            rows = table.find_elements(By.TAG_NAME, "tr") 
            print(f"Found {len(rows)} rows on this page.")

            for row in rows:
                cells = row.find_elements(By.TAG_NAME, "td") 
                if cells: 
                    row_data = [cell.text.strip() for cell in cells]  
                    print(f"Row data: {row_data}") 
                    data.append(row_data)

        except NoSuchElementException:
            print("No table found on the page.")
            break

        try:
            next_button = driver.find_element(By.XPATH, "/html/body/div/div[2]/div[2]/div[1]/div/div[3]/ul/li[2]/i[2]")
            
            if "disabled" in next_button.get_attribute("class"):
                print("No more pages to scrape.")
                break 
                
            driver.execute_script("arguments[0].click();", next_button)  # Click using JavaScript
            print("Navigating to the next page...")
            sleep(3) 

        except NoSuchElementException:
            print("No 'Next' button found. Exiting pagination.")
            break

except Exception as e:
    print("Error scraping table data:", e)

print("\nScraped Data:")
for row in data:
    print(row)

driver.quit()


Scraping current page...
Found 12 rows on this page.
Row data: ['20241226040', '14760', '0', '']
Row data: ['20241226039', '14768', '8', '']
Row data: ['20241226038', '14783', '3', '']
Row data: ['20241226037', '14775', '5', '']
Row data: ['20241226036', '14790', '0', '']
Row data: ['20241226035', '14784', '4', '']
Row data: ['20241226034', '14767', '7', '']
Row data: ['20241226033', '14753', '3', '']
Row data: ['20241226032', '14782', '2', '']
Row data: ['20241226031', '14804', '4', '']
Navigating to the next page...
Scraping current page...
Found 12 rows on this page.
Row data: ['20241226030', '14828', '8', '']
Row data: ['20241226029', '14854', '4', '']
Row data: ['20241226028', '14877', '7', '']
Row data: ['20241226027', '14895', '5', '']
Row data: ['20241226026', '14918', '8', '']
Row data: ['20241226025', '14920', '0', '']
Row data: ['20241226024', '14920', '0', '']
Row data: ['20241226023', '14896', '6', '']
Row data: ['20241226022', '14924', '4', '']
Row data: ['20241226021', '

In [9]:

import pandas as pd

columns = ['Column1', 'Column2', 'Column3', 'FinalColumn'] 
df = pd.DataFrame(data, columns=columns)

df['Column3'] = pd.to_numeric(df['Column3'], errors='coerce')

def update_final_column(value):
    if value % 2 == 0:  
        return 1
    else:               
        return 2

# Apply the function to the numeric column (Column3)
df['FinalColumn'] = df['Column3'].apply(update_final_column)

# Save or display the updated DataFrame
df.to_csv('updated_table_data.csv', index=False)
print(df)


          Column1 Column2  Column3  FinalColumn
0     20241226040   14760        0            1
1     20241226039   14768        8            1
2     20241226038   14783        3            2
3     20241226037   14775        5            2
4     20241226036   14790        0            1
...           ...     ...      ...          ...
4555  20241217353   14841        1            2
4556  20241217352   14857        7            2
4557  20241217351   14846        6            1
4558  20241217350   14838        8            1
4559  20241217349   14811        1            2

[4560 rows x 4 columns]


In [35]:
import numpy as np
import pandas as pd

np.random.seed(42)

# Generate mock data
n_samples = 1000
column1 = np.random.randint(1, 100, size=n_samples)
column2 = np.random.randint(1, 50, size=n_samples)
column3 = np.random.randint(1, 20, size=n_samples)

final_column = [1 if value % 2 == 0 else 2 for value in column3]

reconstructed_df = pd.DataFrame({
    'Column1': column1,
    'Column2': column2,
    'Column3': column3,
    'FinalColumn': final_column
})

reconstructed_df.to_csv("reconstructed_dataset.csv", index=False)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
import joblib

df = pd.read_csv('reconstructed_dataset.csv')

df['Column1'] = pd.to_numeric(df['Column1'], errors='coerce')

X = df[['Column1']]  
y = df['FinalColumn'] - 1  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"XGBoost Model Accuracy: {accuracy * 100:.2f}%")

joblib.dump(xgb_model, 'xgboost_final_column_predictor.pkl')

classification_report_text = classification_report(y_test, y_pred, target_names=['Class 1', 'Class 2'])
print("\nClassification Report:\n", classification_report_text)

def predict_final_column():
    """Predict the FinalColumn value based on user input."""
    
    try:
        model = joblib.load('xgboost_final_column_predictor.pkl')
    except FileNotFoundError:
        print("Trained model file not found. Ensure the model is trained and saved correctly.")
        return

    try:
        user_input = float(input("Enter the value for Column1: "))
        user_data = pd.DataFrame({'Column1': [user_input]})
        prediction = model.predict(user_data)
       
        print(f"The predicted FinalColumn value is: {prediction[0] + 1}")
    except ValueError:
        print("Invalid input. Please enter a numeric value for Column1.")

if __name__ == "__main__":
    predict_final_column()


C:\Users\sanja\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [03:20:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Model Accuracy: 49.50%

Classification Report:
               precision    recall  f1-score   support

     Class 1       0.46      0.45      0.45        93
     Class 2       0.53      0.53      0.53       107

    accuracy                           0.49       200
   macro avg       0.49      0.49      0.49       200
weighted avg       0.49      0.49      0.49       200

